# LABORATORIO 3
# Luz Arévalo
# 20200392

## Parte 1

#### Conexión

In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


#### Pregunta 1

In [2]:
P_1 = """
SELECT *
FROM(
    SELECT
        A.emp_no,
        A.first_name,
        A.last_name,
        C.title,
        B.salary,
        B.from_date,
        B.to_date
    FROM employee.employees A
    LEFT JOIN employee.salaries B ON B.emp_no = A.emp_no
    LEFT JOIN employee.titles C ON C.emp_no = A.emp_no
    GROUP BY 1,2,3,4,5,6,7
    ) A
WHERE emp_no = 12557
"""

pd.read_sql(P_1,connection)

,emp_no,first_name,last_name,title,salary,from_date,to_date
0,12557,Stafford,Kopetz,Senior Engineer,41840,1988-03-20,1989-03-20
1,12557,Stafford,Kopetz,Senior Engineer,46021,1989-03-20,1990-03-20
2,12557,Stafford,Kopetz,Senior Engineer,48699,1990-03-20,1991-03-20
3,12557,Stafford,Kopetz,Senior Engineer,51351,1992-03-19,1993-03-19
4,12557,Stafford,Kopetz,Senior Engineer,51610,1991-03-20,1992-03-19
5,12557,Stafford,Kopetz,Senior Engineer,51660,1993-03-19,1994-03-19
6,12557,Stafford,Kopetz,Senior Engineer,52878,1994-03-19,1995-03-19
7,12557,Stafford,Kopetz,Senior Engineer,54053,1995-03-19,1996-03-18
8,12557,Stafford,Kopetz,Senior Engineer,57950,1996-03-18,1997-03-18
9,12557,Stafford,Kopetz,Senior Engineer,58055,1997-03-18,1998-03-18


#### Pregunta 2

In [3]:
P_2 = """
SELECT * 
FROM
    (SELECT
        A.first_name,
        A.last_name,
        A.emp_no,
        A.birth_date,
        B.dept_no,
        C.dept_name,
        B.from_date,
        B.to_date,
        ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.from_date DESC) AS sec
    FROM employee.employees A
    LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
    LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
    GROUP BY 1,2,3,4,5,6,7,8)A
WHERE emp_no = 12557 and sec = 1
"""
pd.read_sql(P_2,connection)

,first_name,last_name,emp_no,birth_date,dept_no,dept_name,from_date,to_date,sec
0,Stafford,Kopetz,12557,1954-01-01,d004,Production,1994-09-23,9999-01-01,1


#### Pregunta 3

In [4]:
P_3 = """
SELECT * FROM
(SELECT
        A.emp_no,
        A.first_name,
        A.last_name,
        A.birth_date,
        B.dept_no,
        C.dept_name,
        B.from_date as f_date_dept,
        B.to_date as t_date_dept,
        D.from_date,
        D.to_date,
        D.salary,
        ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY D.salary DESC) AS sec
FROM employee.employees A
LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
LEFT JOIN employee.salaries D ON D.emp_no = A.emp_no AND(D.to_date <= B.to_date AND D.from_date >= B.from_date)
GROUP BY 1,2,3,4,5,6,7,8,9,10,11) A
WHERE sec = 1
LIMIT 10
"""

pd.read_sql(P_3,connection)

,emp_no,first_name,last_name,birth_date,dept_no,dept_name,f_date_dept,t_date_dept,from_date,to_date,salary,sec
0,10001,Georgi,Facello,1953-09-02,d005,Development,1986-06-26,9999-01-01,2002-06-22,9999-01-01,88958,1
1,10002,Bezalel,Simmel,1964-06-02,d007,Sales,1996-08-03,9999-01-01,2001-08-02,9999-01-01,72527,1
2,10003,Parto,Bamford,1959-12-03,d004,Production,1995-12-03,9999-01-01,2000-12-01,2001-12-01,43699,1
3,10004,Chirstian,Koblick,1954-05-01,d004,Production,1986-12-01,9999-01-01,2001-11-27,9999-01-01,74057,1
4,10005,Kyoichi,Maliniak,1955-01-21,d003,Human Resources,1989-09-12,9999-01-01,2001-09-09,9999-01-01,94692,1
5,10006,Anneke,Preusig,1953-04-20,d005,Development,1990-08-05,9999-01-01,2000-08-02,2001-08-02,60098,1
6,10007,Tzvetan,Zielinski,1957-05-23,d008,Research,1989-02-10,9999-01-01,2002-02-07,9999-01-01,88070,1
7,10008,Saniya,Kalloufi,1958-02-19,d005,Development,1998-03-11,2000-07-31,2000-03-10,2000-07-31,52668,1
8,10009,Sumant,Peac,1952-04-19,d006,Quality Management,1985-02-18,9999-01-01,2001-02-14,2002-02-14,94443,1
9,10010,Duangkaew,Piveteau,1963-06-01,d006,Quality Management,2000-06-26,9999-01-01,2001-11-23,9999-01-01,80324,1


## Parte 2

#### Conexión

In [5]:
import pandas as pd
import sqlite3 as sq
con = sq.connect('sql-murder-mystery.db')

#### Estructura en la base de datos

In [6]:
a = con.execute("SELECT name, sql FROM sqlite_master WHERE type = 'table';")
for i in a:
    print('Tabla:',i[0],'\nStr:',i[1])

Tabla: crime_scene_report 
Str: CREATE TABLE crime_scene_report (
        date integer,
        type text,
        description text,
        city text
    )
Tabla: drivers_license 
Str: CREATE TABLE drivers_license (
        id integer PRIMARY KEY,
        age integer,
        height integer,
        eye_color text,
        hair_color text,
        gender text,
        plate_number text,
        car_make text,
        car_model text
    )
Tabla: person 
Str: CREATE TABLE person (
        id integer PRIMARY KEY,
        name text,
        license_id integer,
        address_number integer,
        address_street_name text,
        ssn integer,
        FOREIGN KEY (license_id) REFERENCES drivers_license(id)
    )
Tabla: facebook_event_checkin 
Str: CREATE TABLE facebook_event_checkin (
        person_id integer,
        event_id integer,
        event_name text,
        date integer,
        FOREIGN KEY (person_id) REFERENCES person(id)
    )
Tabla: interview 
Str: CREATE TABLE interview

#### Reporte de la escena del crimen

Recuerdo que el crimen fue un asesinato, que ocurrió en algún momento del 15 de enero de 2018 y que tuvo lugar en SQL City.

In [7]:
rsc ="""SELECT * 
FROM crime_scene_report 
LIMIT 2;"""
pd.read_sql(rsc, con)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany


In [8]:
reporte = """
SELECT *
FROM crime_scene_report 
WHERE type = 'murder' 
AND(date = 20180115 AND city = 'SQL City');
"""
a = con.execute(reporte)
for i in a:
    print(i[2])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


#### Debo revisar la declaración de los testigos

In [9]:
testigo1 = """
SELECT *
FROM (SELECT
            person.id,
            person.name,
            person.address_number,
            person.address_street_name,
            interview.transcript
        FROM person
        LEFT JOIN interview ON interview.person_id = person.id) A
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1;
"""
a = con.execute(testigo1)
for i in a:
    print(i[4])

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".


In [10]:
testigo2 = """
SELECT *
FROM (SELECT
            person.id,
            person.name,
            person.address_number,
            person.address_street_name,
            interview.transcript
        FROM person
        LEFT JOIN interview ON interview.person_id = person.id) A
WHERE address_street_name = 'Franklin Ave' AND (name LIKE 'Annabel%');
"""
a = con.execute(testigo2)
for i in a:
    print(i[4])

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


#### Corroboraré información de los testigos
    Membresía: '48Z%'
    Placas: '%H42W%'
    Visto : Enero 2018

In [11]:
dt1 = """
SELECT *
FROM (SELECT
            person.id,
            person.name,
            person.license_id,            
            get_fit_now_member.id AS id_member,
            drivers_license.plate_number AS plate,
            get_fit_now_check_in.check_in_date
        FROM person
        LEFT JOIN get_fit_now_member ON get_fit_now_member.person_id = person.id
        LEFT JOIN drivers_license ON drivers_license.id = person.license_id
        LEFT JOIN get_fit_now_check_in ON get_fit_now_member.id = get_fit_now_check_in.membership_id) A
WHERE id_member LIKE '48Z%' AND(plate LIKE '%H42W%' AND check_in_date LIKE '201801%');
"""
pd.read_sql(dt1, con)

,id,name,license_id,id_member,plate,check_in_date
0,67318,Jeremy Bowers,423327,48Z55,0H42W2,20180109


In [12]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]


"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [16]:
murder_interview = """
SELECT *
FROM (SELECT
            person.id,
            interview.transcript
        FROM person
        LEFT JOIN interview ON interview.person_id = person.id) A
WHERE id = 67318;
"""
a = con.execute(murder_interview)
for i in a:
    print(i[1])

I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



In [14]:
villian = """
SELECT *
FROM (SELECT
            person.name,
            drivers_license.gender,
            drivers_license.height,
            drivers_license.hair_color,
            drivers_license.car_make,
            drivers_license.car_model,
            facebook_event_checkin.event_name,
            facebook_event_checkin.date
        FROM person
        LEFT JOIN get_fit_now_member ON get_fit_now_member.person_id = person.id
        LEFT JOIN drivers_license ON drivers_license.id = person.license_id
        LEFT JOIN facebook_event_checkin ON facebook_event_checkin.person_id = person.id) A
WHERE gender = 'female' 
        AND(height BETWEEN 55 AND 67)
        AND hair_color = 'red' 
        AND(car_make = 'Tesla' AND car_model = 'Model S') 
        AND event_name = 'SQL Symphony Concert'
        AND date LIKE '2017%12'
        ;
"""
pd.read_sql(villian, con)

,name,gender,height,hair_color,car_make,car_model,event_name,date
0,Miranda Priestly,female,66,red,Tesla,Model S,SQL Symphony Concert,20171212


In [15]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'